In [5]:
import concurrent.futures
from nba_api.stats.endpoints import boxscoretraditionalv2, leaguegamelog
import pandas as pd
import time
import requests

# Function to fetch game logs with retries
def fetch_game_logs(season, retries=5, delay=10):
    session = requests.Session()
    session.headers.update({
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Accept': 'application/json, text/plain, */*',
        'Referer': 'https://stats.nba.com/',
        'Origin': 'https://stats.nba.com',
        'Connection': 'keep-alive',
    })
    for attempt in range(retries):
        try:
            game_logs = leaguegamelog.LeagueGameLog(season=season, headers=session.headers).get_data_frames()[0]
            print("Fetched game logs successfully.")
            return game_logs
        except Exception as e:
            print(f"Attempt {attempt + 1} - An error occurred while fetching game logs: {e}")
            if attempt < retries - 1:
                time.sleep(delay)
    return pd.DataFrame()  # Return an empty DataFrame if all retries fail

# Fetch all games played in the 2022-23 season
season = '2022-23'
game_logs = fetch_game_logs(season)

# Check if game_logs DataFrame is not empty
if not game_logs.empty:
    # Initialize a DataFrame to store all player box scores
    all_box_scores = pd.DataFrame()

    # Function to fetch and return player box scores
    def fetch_box_score(game_id):
        try:
            print(f"Fetching box score for game ID: {game_id}")
            box_score = boxscoretraditionalv2.BoxScoreTraditionalV2(game_id=game_id).get_data_frames()[0]
            return box_score
        except Exception as e:
            print(f"An error occurred while fetching box score for game ID {game_id}: {e}")
            return pd.DataFrame()  # Return an empty DataFrame in case of error

    # Use ThreadPoolExecutor to fetch box scores in parallel
    with concurrent.futures.ThreadPoolExecutor(max_workers=20) as executor:
        future_to_game_id = {executor.submit(fetch_box_score, game.GAME_ID): game.GAME_ID for game in game_logs.itertuples()}
        
        for future in concurrent.futures.as_completed(future_to_game_id):
            game_id = future_to_game_id[future]
            try:
                box_score = future.result()
                if not box_score.empty:  # Only concatenate non-empty DataFrames
                    all_box_scores = pd.concat([all_box_scores, box_score], ignore_index=True)
                    print(f"Completed fetching box score for game ID: {game_id}")
            except Exception as e:
                print(f"An error occurred while processing future for game ID {game_id}: {e}")

    # Display the first few rows of the combined box scores
    display(all_box_scores.head())

    # Save the combined box scores to a CSV file
    all_box_scores.to_csv('all_player_box_scores_2022_2023.csv', index=False)
    print("Player box scores saved successfully!")
else:
    print("No game logs available for the specified season.")


Fetched game logs successfully.
Fetching box score for game ID: 0022200001
Fetching box score for game ID: 0022200001
Fetching box score for game ID: 0022200002
Fetching box score for game ID: 0022200002
Fetching box score for game ID: 0022200005
Fetching box score for game ID: 0022200004
Fetching box score for game ID: 0022200009
Fetching box score for game ID: 0022200009
Fetching box score for game ID: 0022200008
Fetching box score for game ID: 0022200008
Fetching box score for game ID: 0022200010
Fetching box score for game ID: 0022200010
Fetching box score for game ID: 0022200013
Fetching box score for game ID: 0022200013
Fetching box score for game ID: 0022200011
Fetching box score for game ID: 0022200011
Fetching box score for game ID: 0022200005
Fetching box score for game ID: 0022200004
Fetching box score for game ID: 0022200003
Fetching box score for game ID: 0022200003
Fetching box score for game ID: 0022200006
Completed fetching box score for game ID: 0022200002
Fetching box

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,NICKNAME,START_POSITION,COMMENT,MIN,...,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS
0,0022200002,1610612747,LAL,Los Angeles,1629022,Lonnie Walker IV,Lonnie,F,,29.000000:17,...,1.0,2.0,3.0,5.0,1.0,1.0,2.0,0.0,5.0,-13.0
1,0022200002,1610612747,LAL,Los Angeles,2544,LeBron James,LeBron,F,,35.000000:12,...,5.0,10.0,15.0,8.0,0.0,0.0,5.0,2.0,31.0,-10.0
2,0022200002,1610612747,LAL,Los Angeles,203076,Anthony Davis,Anthony,C,,35.000000:36,...,0.0,6.0,6.0,0.0,4.0,1.0,3.0,2.0,27.0,-21.0
3,0022200002,1610612747,LAL,Los Angeles,201566,Russell Westbrook,Russell,G,,30.000000:41,...,1.0,10.0,11.0,3.0,1.0,0.0,4.0,1.0,19.0,-6.0
4,0022200002,1610612747,LAL,Los Angeles,201976,Patrick Beverley,Patrick,G,,24.000000:40,...,1.0,1.0,2.0,2.0,4.0,2.0,1.0,5.0,3.0,-5.0


Player box scores saved successfully!
